In [1]:
import pandas as pd
import numpy as np
from pprint import pprint
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
os.chdir('d:\\vscode_machineLearning\\internship\\sentiment-Analysis-fellowship.ai')
import random

In [2]:
df = pd.read_csv('sentiment-analysis-dataset/IMDB_clean_data.csv',index_col=False)
df = df.drop('Unnamed: 0',axis=1) # droping the Unnamed: 0
df.dropna(inplace=True) # I had one NaN value in my dataframe.
df = df.reset_index()

In [3]:
padded_docs = pd.read_csv('sentiment-analysis-dataset/padded_docs.csv')
padded_docs = padded_docs.drop(columns='Unnamed: 0')

In [21]:
padded_docs = padded_docs.iloc[:,:150]

In [22]:
padded_docs = np.array(padded_docs)

## train test split

In [23]:
X = df ['review']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(
     np.array(padded_docs), np.array(y), test_size=0.23, random_state=42)

In [24]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (38499, 150)
X_test shape: (11500, 150)
y_train shape: (38499,)
y_test shape: (11500,)


## model training

In [25]:
from keras.layers import BatchNormalization, Dropout, Bidirectional, LSTM, Embedding, Dense 
from keras.losses import binary_crossentropy
from tensorflow import keras
from keras.callbacks import LearningRateScheduler , EarlyStopping
from keras.activations import relu , sigmoid


In [32]:
model_features = 100 # for embedding layer
input_len = 150
model = keras.Sequential(name='LSTM_model')

model.add(Embedding(
    input_dim=56942,  # The "+1" accounts for the reserved index 0 in the word index (since word indices start from 1 and not 0)
    output_dim=model_features, input_length=input_len,name = 'input_layer'
))
model.add(Bidirectional(
    LSTM(units=64,activation=relu,return_sequences=True),
    name='LSTM_1'
))
model.add(
    Dropout(rate=0.5))
model.add(
    BatchNormalization())
model.add(
    Dropout(rate=0.5))
model.add(Bidirectional(
    LSTM(units=32,activation=relu,return_sequences=False),
    name='LSTM_2'
))
model.add(Dense(
    units=128,activation=relu,name='fully_connected_layer'
))
model.add(Dense(
    units=1,activation=sigmoid,name='output_layer'
))

In [33]:
model.summary()

Model: "LSTM_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (Embedding)     (None, 150, 100)          5694200   
                                                                 
 LSTM_1 (Bidirectional)      (None, 150, 128)          84480     
                                                                 
 dropout_2 (Dropout)         (None, 150, 128)          0         
                                                                 
 batch_normalization_1 (Batc  (None, 150, 128)         512       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 150, 128)          0         
                                                                 
 LSTM_2 (Bidirectional)      (None, 64)                41216     
                                                        

In [34]:
def lr_schedule(epoch, lr):
    if epoch < 1:
        return lr
    else:
        return lr * np.exp(-0.1)

# learning rate scheduler callback to descrese the learning rate gradually as the epochs increases So that my alogrithm could not jump out of Global minima.
lr_scheduler = LearningRateScheduler(lr_schedule)

# Early stopping to stop the Neural Network when we get same Validation accuracy
early_stopping = EarlyStopping(
    monitor="accuracy",
    min_delta=0.00001,
    patience=5,
    verbose=1,
    mode="auto",
    baseline=None,
    restore_best_weights=False
)

In [35]:
optimizer = keras.optimizers.Adam(learning_rate=0.0005)

In [36]:
model.compile(optimizer=optimizer, # Used Adam because this has not any major disadvantages with custom learning rate because the convergence was very unstable.
               loss=binary_crossentropy, # because solving the classification problem
                 metrics=['accuracy'])  # I don't need to write about this you know.

In [37]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20,
                    batch_size=32, # I had tried different batch sizes but this has given my best results
                      callbacks=[lr_scheduler, early_stopping]) # these to prevent the NN from overfitting and scheduling learning rate to get optimum solution.

Epoch 1/20
1204/1204 [==============================] - 496s 403ms/step - loss: 40354640.0000 - accuracy: 0.6660 - val_loss: 0.5525 - val_accuracy: 0.7262 - lr: 5.0000e-04
Epoch 2/20
1204/1204 [==============================] - 445s 370ms/step - loss: 56.9634 - accuracy: 0.7304 - val_loss: 0.5775 - val_accuracy: 0.7030 - lr: 4.5242e-04
Epoch 3/20
1204/1204 [==============================] - 444s 368ms/step - loss: 0.5106 - accuracy: 0.7487 - val_loss: 0.5684 - val_accuracy: 0.7084 - lr: 4.0937e-04
Epoch 4/20
1204/1204 [==============================] - 487s 404ms/step - loss: 0.4793 - accuracy: 0.7649 - val_loss: 0.5720 - val_accuracy: 0.7023 - lr: 3.7041e-04
Epoch 5/20
1204/1204 [==============================] - 481s 400ms/step - loss: 0.4575 - accuracy: 0.7799 - val_loss: 0.5753 - val_accuracy: 0.7010 - lr: 3.3516e-04
Epoch 6/20
1204/1204 [==============================] - 487s 405ms/step - loss: 0.6648 - accuracy: 0.7898 - val_loss: 0.5773 - val_accuracy: 0.7034 - lr: 3.0327e-04
Ep

In [38]:
model.save('sentimentAnalysisModel.H5')

INFO:tensorflow:Assets written to: sentimentAnalysisModel.H5\assets


INFO:tensorflow:Assets written to: sentimentAnalysisModel.H5\assets
